# Encoder-Decoder Model with Attention

In [35]:
from random import randint
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, RepeatVector

Using TensorFlow backend.


In [2]:
def generate_sequence(length, n_unique):
    return [randint(0, n_unique - 1) for _ in range(length)]

In [13]:
def one_hot_encode(sequence, n_unique):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(n_unique)]
        vector[value] = 1
        encoding.append(vector)
    return np.array(encoding)

In [14]:
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [16]:
sequence = generate_sequence(5, 50)
print(sequence)
encoded = one_hot_encode(sequence, 50)
print(encoded.shape)
print(encoded)
decoded = one_hot_decode(encoded)
print(decoded)

[27, 22, 3, 5, 4]
(5, 50)
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0]]
[27, 22, 3, 5, 4]


In [28]:
def get_pair(n_in, n_out, n_unique):
    # n_in=5, n_out=2のとき
    # 入力系列と出力系列は同じ長さで0パディングする
    # [4, 8, 1, 2, 4] => [4, 8, 0, 0, 0]
    sequence_in = generate_sequence(n_in, n_unique)
    sequence_out = sequence_in[:n_out] + [0 for _ in range(n_in - n_out)]
    
    X = one_hot_encode(sequence_in, n_unique)
    y = one_hot_encode(sequence_out, n_unique)
    
    # (samples, time steps, features)
    X = X.reshape((1, X.shape[0], X.shape[1]))
    y = y.reshape((1, y.shape[0], y.shape[1]))
    
    return X, y

In [32]:
X, y = get_pair(5, 2, 50)
print(X.shape, y.shape)
print('X=%s, y=%s' % (one_hot_decode(X[0]), one_hot_decode(y[0])))

(1, 5, 50) (1, 5, 50)
X=[26, 42, 49, 24, 21], y=[26, 42, 0, 0, 0]


## Encoder-Decoder without Attention

In [33]:
n_features = 50
n_timesteps_in = 5
n_timesteps_out = 2

In [36]:
model = Sequential()
# return_sequences=Falseなので最後の入力の出力のみ返す
model.add(LSTM(150, input_shape=(n_timesteps_in, n_features)))
model.add(RepeatVector(n_timesteps_in))  # 出力は入力系列と同じ系列長だけ出力する
model.add(LSTM(150, return_sequences=True))
model.add(TimeDistributed(Dense(n_features, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [37]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 150)               120600    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 150)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 150)            180600    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 50)             7550      
Total params: 308,750
Trainable params: 308,750
Non-trainable params: 0
_________________________________________________________________


In [38]:
for epoch in range(5000):
    X, y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
    model.fit(X, y, epochs=1, verbose=2)

Epoch 1/1
0s - loss: 3.9090
Epoch 1/1
0s - loss: 3.9098
Epoch 1/1
0s - loss: 3.8977
Epoch 1/1
0s - loss: 3.8848
Epoch 1/1
0s - loss: 3.8890
Epoch 1/1
0s - loss: 3.8548
Epoch 1/1
0s - loss: 3.8532
Epoch 1/1
0s - loss: 3.8297
Epoch 1/1
0s - loss: 3.8393
Epoch 1/1
0s - loss: 3.8151
Epoch 1/1
0s - loss: 3.7643
Epoch 1/1
0s - loss: 3.7800
Epoch 1/1
0s - loss: 3.7405
Epoch 1/1
0s - loss: 3.7108
Epoch 1/1
0s - loss: 3.6442
Epoch 1/1
0s - loss: 3.6559
Epoch 1/1
0s - loss: 3.5473
Epoch 1/1
0s - loss: 3.4392
Epoch 1/1
0s - loss: 3.3717
Epoch 1/1
0s - loss: 3.2781
Epoch 1/1
0s - loss: 3.1957
Epoch 1/1
0s - loss: 3.0078
Epoch 1/1
0s - loss: 2.5106
Epoch 1/1
0s - loss: 2.2233
Epoch 1/1
0s - loss: 2.1107
Epoch 1/1
0s - loss: 1.8266
Epoch 1/1
0s - loss: 1.8224
Epoch 1/1
0s - loss: 1.7138
Epoch 1/1
0s - loss: 1.7005
Epoch 1/1
0s - loss: 1.6827
Epoch 1/1
0s - loss: 1.5952
Epoch 1/1
0s - loss: 1.7757
Epoch 1/1
0s - loss: 1.7542
Epoch 1/1
0s - loss: 1.8469
Epoch 1/1
0s - loss: 2.0296
Epoch 1/1
0s - loss:

0s - loss: 1.6308
Epoch 1/1
0s - loss: 1.6001
Epoch 1/1
0s - loss: 1.6445
Epoch 1/1
0s - loss: 1.6270
Epoch 1/1
0s - loss: 1.5270
Epoch 1/1
0s - loss: 1.6471
Epoch 1/1
0s - loss: 1.6220
Epoch 1/1
0s - loss: 1.5731
Epoch 1/1
0s - loss: 1.5636
Epoch 1/1
0s - loss: 1.5683
Epoch 1/1
0s - loss: 1.6096
Epoch 1/1
0s - loss: 1.5436
Epoch 1/1
0s - loss: 1.6086
Epoch 1/1
0s - loss: 1.5736
Epoch 1/1
0s - loss: 1.6583
Epoch 1/1
0s - loss: 1.5829
Epoch 1/1
0s - loss: 1.5646
Epoch 1/1
0s - loss: 1.5832
Epoch 1/1
0s - loss: 1.5822
Epoch 1/1
0s - loss: 1.5883
Epoch 1/1
0s - loss: 1.6027
Epoch 1/1
0s - loss: 1.4691
Epoch 1/1
0s - loss: 1.5991
Epoch 1/1
0s - loss: 1.5820
Epoch 1/1
0s - loss: 1.6175
Epoch 1/1
0s - loss: 1.6120
Epoch 1/1
0s - loss: 1.5481
Epoch 1/1
0s - loss: 1.5466
Epoch 1/1
0s - loss: 1.6809
Epoch 1/1
0s - loss: 1.6036
Epoch 1/1
0s - loss: 1.6899
Epoch 1/1
0s - loss: 1.6611
Epoch 1/1
0s - loss: 1.2296
Epoch 1/1
0s - loss: 1.5731
Epoch 1/1
0s - loss: 1.5650
Epoch 1/1
0s - loss: 1.5534
Ep

0s - loss: 1.4858
Epoch 1/1
0s - loss: 1.6539
Epoch 1/1
0s - loss: 1.6376
Epoch 1/1
0s - loss: 1.5441
Epoch 1/1
0s - loss: 1.5955
Epoch 1/1
0s - loss: 1.5684
Epoch 1/1
0s - loss: 1.5567
Epoch 1/1
0s - loss: 1.5562
Epoch 1/1
0s - loss: 1.5668
Epoch 1/1
0s - loss: 1.5825
Epoch 1/1
0s - loss: 1.5507
Epoch 1/1
0s - loss: 1.5578
Epoch 1/1
0s - loss: 1.5446
Epoch 1/1
0s - loss: 1.5179
Epoch 1/1
0s - loss: 1.5306
Epoch 1/1
0s - loss: 1.6233
Epoch 1/1
0s - loss: 1.5204
Epoch 1/1
0s - loss: 1.6139
Epoch 1/1
0s - loss: 1.5120
Epoch 1/1
0s - loss: 1.4841
Epoch 1/1
0s - loss: 1.5119
Epoch 1/1
0s - loss: 1.5958
Epoch 1/1
0s - loss: 1.1520
Epoch 1/1
0s - loss: 1.5253
Epoch 1/1
0s - loss: 1.5073
Epoch 1/1
0s - loss: 1.5924
Epoch 1/1
0s - loss: 1.4823
Epoch 1/1
0s - loss: 1.6280
Epoch 1/1
0s - loss: 1.6145
Epoch 1/1
0s - loss: 1.5361
Epoch 1/1
0s - loss: 1.5417
Epoch 1/1
0s - loss: 1.5436
Epoch 1/1
0s - loss: 1.4906
Epoch 1/1
0s - loss: 1.6142
Epoch 1/1
0s - loss: 1.5490
Epoch 1/1
0s - loss: 1.4747
Ep

0s - loss: 1.5513
Epoch 1/1
0s - loss: 1.4118
Epoch 1/1
0s - loss: 1.5742
Epoch 1/1
0s - loss: 1.2987
Epoch 1/1
0s - loss: 1.5858
Epoch 1/1
0s - loss: 1.5955
Epoch 1/1
0s - loss: 1.6504
Epoch 1/1
0s - loss: 0.8886
Epoch 1/1
0s - loss: 1.2453
Epoch 1/1
0s - loss: 1.3730
Epoch 1/1
0s - loss: 1.5198
Epoch 1/1
0s - loss: 1.6771
Epoch 1/1
0s - loss: 1.5279
Epoch 1/1
0s - loss: 1.1501
Epoch 1/1
0s - loss: 1.0589
Epoch 1/1
0s - loss: 1.6486
Epoch 1/1
0s - loss: 1.4788
Epoch 1/1
0s - loss: 1.6480
Epoch 1/1
0s - loss: 1.1357
Epoch 1/1
0s - loss: 1.5673
Epoch 1/1
0s - loss: 1.2775
Epoch 1/1
0s - loss: 1.6214
Epoch 1/1
0s - loss: 1.5864
Epoch 1/1
0s - loss: 1.6218
Epoch 1/1
0s - loss: 1.4932
Epoch 1/1
0s - loss: 1.4726
Epoch 1/1
0s - loss: 1.2859
Epoch 1/1
0s - loss: 1.6591
Epoch 1/1
0s - loss: 1.5972
Epoch 1/1
0s - loss: 1.4363
Epoch 1/1
0s - loss: 1.4458
Epoch 1/1
0s - loss: 1.6008
Epoch 1/1
0s - loss: 1.5663
Epoch 1/1
0s - loss: 1.5687
Epoch 1/1
0s - loss: 1.3791
Epoch 1/1
0s - loss: 1.5852
Ep

0s - loss: 1.4197
Epoch 1/1
0s - loss: 1.5349
Epoch 1/1
0s - loss: 1.4363
Epoch 1/1
0s - loss: 1.3001
Epoch 1/1
0s - loss: 1.4401
Epoch 1/1
0s - loss: 1.2961
Epoch 1/1
0s - loss: 1.7935
Epoch 1/1
0s - loss: 1.3320
Epoch 1/1
0s - loss: 1.6902
Epoch 1/1
0s - loss: 1.1097
Epoch 1/1
0s - loss: 1.4112
Epoch 1/1
0s - loss: 1.1705
Epoch 1/1
0s - loss: 1.4362
Epoch 1/1
0s - loss: 1.3770
Epoch 1/1
0s - loss: 1.5246
Epoch 1/1
0s - loss: 1.6656
Epoch 1/1
0s - loss: 1.1917
Epoch 1/1
0s - loss: 1.6110
Epoch 1/1
0s - loss: 1.3566
Epoch 1/1
0s - loss: 1.5339
Epoch 1/1
0s - loss: 1.6408
Epoch 1/1
0s - loss: 1.3710
Epoch 1/1
0s - loss: 1.1496
Epoch 1/1
0s - loss: 1.3860
Epoch 1/1
0s - loss: 1.6986
Epoch 1/1
0s - loss: 1.1681
Epoch 1/1
0s - loss: 1.2929
Epoch 1/1
0s - loss: 1.0092
Epoch 1/1
0s - loss: 1.4840
Epoch 1/1
0s - loss: 1.5726
Epoch 1/1
0s - loss: 1.3988
Epoch 1/1
0s - loss: 1.3327
Epoch 1/1
0s - loss: 1.2810
Epoch 1/1
0s - loss: 1.5026
Epoch 1/1
0s - loss: 1.4366
Epoch 1/1
0s - loss: 1.3387
Ep

...

Epoch 1/1
0s - loss: 0.8838
Epoch 1/1
0s - loss: 0.9370
Epoch 1/1
0s - loss: 1.1464
Epoch 1/1
0s - loss: 0.3414
Epoch 1/1
0s - loss: 0.8967
Epoch 1/1
0s - loss: 0.6266
Epoch 1/1
0s - loss: 0.7046
Epoch 1/1
0s - loss: 1.0300
Epoch 1/1
0s - loss: 0.5068
Epoch 1/1
0s - loss: 0.4588
Epoch 1/1
0s - loss: 0.4573
Epoch 1/1
0s - loss: 0.5374
Epoch 1/1
0s - loss: 0.5509
Epoch 1/1
0s - loss: 0.8353
Epoch 1/1
0s - loss: 0.4940
Epoch 1/1
0s - loss: 0.5554
Epoch 1/1
0s - loss: 1.0639
Epoch 1/1
0s - loss: 0.7712
Epoch 1/1
0s - loss: 0.9681
Epoch 1/1
0s - loss: 0.5417
Epoch 1/1
0s - loss: 0.7704
Epoch 1/1
0s - loss: 0.7871
Epoch 1/1
0s - loss: 0.1584
Epoch 1/1
0s - loss: 0.4952
Epoch 1/1
0s - loss: 0.5768
Epoch 1/1
0s - loss: 0.6865
Epoch 1/1
0s - loss: 0.3755
Epoch 1/1
0s - loss: 0.6483
Epoch 1/1
0s - loss: 1.5956
Epoch 1/1
0s - loss: 0.9103
Epoch 1/1
0s - loss: 0.6535
Epoch 1/1
0s - loss: 0.7100
Epoch 1/1
0s - loss: 0.8927
Epoch 1/1
0s - loss: 0.8411
Epoch 1/1
0s - loss: 0.4285
Epoch 1/1
0s - loss:

0s - loss: 0.5089
Epoch 1/1
0s - loss: 0.4748
Epoch 1/1
0s - loss: 0.6160
Epoch 1/1
0s - loss: 0.5166
Epoch 1/1
0s - loss: 0.4622
Epoch 1/1
0s - loss: 0.4990
Epoch 1/1
0s - loss: 0.5977
Epoch 1/1
0s - loss: 0.6321
Epoch 1/1
0s - loss: 0.5795
Epoch 1/1
0s - loss: 0.6125
Epoch 1/1
0s - loss: 1.0563
Epoch 1/1
0s - loss: 0.7583
Epoch 1/1
0s - loss: 0.7976
Epoch 1/1
0s - loss: 0.9902
Epoch 1/1
0s - loss: 0.4933
Epoch 1/1
0s - loss: 0.7107
Epoch 1/1
0s - loss: 0.4582
Epoch 1/1
0s - loss: 0.6507
Epoch 1/1
0s - loss: 0.0916
Epoch 1/1
0s - loss: 0.4449
Epoch 1/1
0s - loss: 0.7676
Epoch 1/1
0s - loss: 0.8145
Epoch 1/1
0s - loss: 0.5902
Epoch 1/1
0s - loss: 0.7657
Epoch 1/1
0s - loss: 0.5779
Epoch 1/1
0s - loss: 0.5693
Epoch 1/1
0s - loss: 0.6725
Epoch 1/1
0s - loss: 0.4262
Epoch 1/1
0s - loss: 0.1469
Epoch 1/1
0s - loss: 0.6044
Epoch 1/1
0s - loss: 0.9540
Epoch 1/1
0s - loss: 1.1172
Epoch 1/1
0s - loss: 0.5308
Epoch 1/1
0s - loss: 0.2295
Epoch 1/1
0s - loss: 0.5960
Epoch 1/1
0s - loss: 0.4272
Ep

0s - loss: 0.4285
Epoch 1/1
0s - loss: 0.2484
Epoch 1/1
0s - loss: 0.8887
Epoch 1/1
0s - loss: 0.5034
Epoch 1/1
0s - loss: 0.9779
Epoch 1/1
0s - loss: 1.2013
Epoch 1/1
0s - loss: 1.2177
Epoch 1/1
0s - loss: 0.2133
Epoch 1/1
0s - loss: 0.4595
Epoch 1/1
0s - loss: 0.4600
Epoch 1/1
0s - loss: 0.7714
Epoch 1/1
0s - loss: 0.7232
Epoch 1/1
0s - loss: 0.6370
Epoch 1/1
0s - loss: 0.4745
Epoch 1/1
0s - loss: 0.9275
Epoch 1/1
0s - loss: 0.6194
Epoch 1/1
0s - loss: 0.3898
Epoch 1/1
0s - loss: 0.1512
Epoch 1/1
0s - loss: 1.0257
Epoch 1/1
0s - loss: 0.5470
Epoch 1/1
0s - loss: 0.6301
Epoch 1/1
0s - loss: 0.5026
Epoch 1/1
0s - loss: 0.8746
Epoch 1/1
0s - loss: 0.7988
Epoch 1/1
0s - loss: 0.7325
Epoch 1/1
0s - loss: 0.2087
Epoch 1/1
0s - loss: 0.3989
Epoch 1/1
0s - loss: 0.4836
Epoch 1/1
0s - loss: 0.4894
Epoch 1/1
0s - loss: 0.4059
Epoch 1/1
0s - loss: 1.3911
Epoch 1/1
0s - loss: 0.8683
Epoch 1/1
0s - loss: 0.3493
Epoch 1/1
0s - loss: 0.5066
Epoch 1/1
0s - loss: 0.6799
Epoch 1/1
0s - loss: 0.6463
Ep

0s - loss: 0.2342
Epoch 1/1
0s - loss: 0.3198
Epoch 1/1
0s - loss: 0.6708
Epoch 1/1
0s - loss: 0.6697
Epoch 1/1
0s - loss: 0.6086
Epoch 1/1
0s - loss: 0.6310
Epoch 1/1
0s - loss: 0.6392
Epoch 1/1
0s - loss: 0.1610
Epoch 1/1
0s - loss: 0.5189
Epoch 1/1
0s - loss: 0.5182
Epoch 1/1
0s - loss: 0.3538
Epoch 1/1
0s - loss: 0.8109
Epoch 1/1
0s - loss: 0.2115
Epoch 1/1
0s - loss: 0.9840
Epoch 1/1
0s - loss: 0.9392
Epoch 1/1
0s - loss: 0.6958
Epoch 1/1
0s - loss: 0.4365
Epoch 1/1
0s - loss: 0.7200
Epoch 1/1
0s - loss: 0.9473
Epoch 1/1
0s - loss: 0.7225
Epoch 1/1
0s - loss: 0.4377
Epoch 1/1
0s - loss: 0.2088
Epoch 1/1
0s - loss: 1.4223
Epoch 1/1
0s - loss: 0.7080
Epoch 1/1
0s - loss: 0.9296
Epoch 1/1
0s - loss: 0.5902
Epoch 1/1
0s - loss: 1.0107
Epoch 1/1
0s - loss: 0.1176
Epoch 1/1
0s - loss: 0.5929
Epoch 1/1
0s - loss: 0.4195
Epoch 1/1
0s - loss: 0.4317
Epoch 1/1
0s - loss: 0.7320
Epoch 1/1
0s - loss: 0.5621
Epoch 1/1
0s - loss: 1.0532
Epoch 1/1
0s - loss: 0.8175
Epoch 1/1
0s - loss: 0.8232
Ep

0s - loss: 0.7950
Epoch 1/1
0s - loss: 0.4168
Epoch 1/1
0s - loss: 0.7512
Epoch 1/1
0s - loss: 0.5135
Epoch 1/1
0s - loss: 0.3180
Epoch 1/1
0s - loss: 0.5394
Epoch 1/1
0s - loss: 0.4675
Epoch 1/1
0s - loss: 0.5252
Epoch 1/1
0s - loss: 0.5916
Epoch 1/1
0s - loss: 0.6461
Epoch 1/1
0s - loss: 0.3440
Epoch 1/1
0s - loss: 0.5709
Epoch 1/1
0s - loss: 0.9200
Epoch 1/1
0s - loss: 0.6643
Epoch 1/1
0s - loss: 0.3649
Epoch 1/1
0s - loss: 0.1211
Epoch 1/1
0s - loss: 0.7566
Epoch 1/1
0s - loss: 0.7314
Epoch 1/1
0s - loss: 0.6378
Epoch 1/1
0s - loss: 0.6835
Epoch 1/1
0s - loss: 0.1615
Epoch 1/1
0s - loss: 0.7441
Epoch 1/1
0s - loss: 0.7058
Epoch 1/1
0s - loss: 0.4765
Epoch 1/1
0s - loss: 0.4843
Epoch 1/1
0s - loss: 0.9092
Epoch 1/1
0s - loss: 0.7760
Epoch 1/1
0s - loss: 0.1844
Epoch 1/1
0s - loss: 1.2227
Epoch 1/1
0s - loss: 0.5669
Epoch 1/1
0s - loss: 0.3778
Epoch 1/1
0s - loss: 0.5037
Epoch 1/1
0s - loss: 0.1374
Epoch 1/1
0s - loss: 0.8603
Epoch 1/1
0s - loss: 0.2686
Epoch 1/1
0s - loss: 0.8435
Ep

In [40]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
    X, y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
    yhat = model.predict(X, verbose=0)
    if np.array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
        correct += 1
print('Accuracy: %.2f%%' % (float(correct) / float(total) * 100.0))

Accuracy: 18.00%


In [41]:
for _ in range(10):
    X, y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
    yhat = model.predict(X, verbose=0)
    print('Expected:', one_hot_decode(y[0]), 'Predicted:', one_hot_decode(yhat[0]))

Expected: [41, 46, 0, 0, 0] Predicted: [41, 41, 0, 0, 0]
Expected: [9, 48, 0, 0, 0] Predicted: [9, 26, 0, 0, 0]
Expected: [6, 1, 0, 0, 0] Predicted: [6, 0, 0, 0, 0]
Expected: [25, 32, 0, 0, 0] Predicted: [25, 32, 0, 0, 0]
Expected: [44, 2, 0, 0, 0] Predicted: [44, 2, 0, 0, 0]
Expected: [46, 31, 0, 0, 0] Predicted: [46, 1, 0, 0, 0]
Expected: [17, 7, 0, 0, 0] Predicted: [17, 14, 0, 0, 0]
Expected: [39, 42, 0, 0, 0] Predicted: [39, 39, 0, 0, 0]
Expected: [6, 24, 0, 0, 0] Predicted: [6, 0, 0, 0, 0]
Expected: [5, 40, 0, 0, 0] Predicted: [5, 5, 0, 0, 0]


## Attention

In [42]:
from attention_decoder import AttentionDecoder